# Parameter selection example

The `sashelp.fish` dataset contains measurements of 159 fish that were caught in Lake Laengelmaevesi in Finland. They are grouped by species.

The dataset contains the following measurements:
* weight ($weight$)
* length from the nose of the fish to the base of the tail ($length_1$)
* length from the nose of the fish to the notch of the tail ($length_2$)
* length from the nose of the fish to the end of the tail ($length_3$)
* height as a percentage of $length_3$ ($height$)
* width as a percentage of $length_3$ ($width$)

We want to determine the relationship between a fish's $weight$ and the rest of its measurements. Our goal is to try and eliminate some of the variables using parameter selection in order to get the most significant subset. Our subsets will be generated using the following requests under the `selection = ` option of the `model` statement in `PROC REG`:
* forward
* backward
* stepwise
* adjrsq
* cp

The `maxr`, `minr`, and `rsquare` requests will not be covered here. `maxr` and `minr` are similar to the `forward`/`backward`/`stepwise` requests, except that it $R^{2}$ improvements lists all possible models. `rsquare` also lists all possible models and is similar to `adjrsq`.

For this example, we will look at the perch species:

In [15]:
proc print data = sashelp.Fish (where = (species = "Perch"));
run;

Obs,Species,Weight,Length1,Length2,Length3,Height,Width
73,Perch,5.9,7.5,8.4,8.8,2.1120,1.4080
74,Perch,32.0,12.5,13.7,14.7,3.5280,1.9992
75,Perch,40.0,13.8,15.0,16.0,3.8240,2.4320
76,Perch,51.5,15.0,16.2,17.2,4.5924,2.6316
77,Perch,70.0,15.7,17.4,18.5,4.5880,2.9415
78,Perch,100.0,16.2,18.0,19.2,5.2224,3.3216
79,Perch,78.0,16.8,18.7,19.4,5.1992,3.1234
80,Perch,80.0,17.2,19.0,20.2,5.6358,3.0502
81,Perch,85.0,17.8,19.6,20.8,5.1376,3.0368
82,Perch,85.0,18.2,20.0,21.0,5.0820,2.7720


Let's adjust the $height$ and $width$ values to be actual measurements and not percentages:

In [16]:
data perch;
    set sashelp.fish;
    where species = "Perch";
    
    height2 = length3 * height/100;
    width2 = length3 * width/100;
    
    drop height width;
    rename height2 = height width2 = width;
run;

Let's first use forward selection:

In [28]:
proc reg data = perch plots = none;
    model weight = length1 length2 length3 height width / selection = forward;
run;

The forward method stops once it can no longer include variables that are statistically significant in the presence of the other variables. In this case, it has determined that the maximum amount of variables in which the model can still be significant is 3, with the feature vaiables being:
* height
* length1
* width

Notice that we started with the $height$ variable and continuously added the others, with each addition reducing the overall F-value of the model. The "Summary of Forward Selection" table gives us a lot of information about which variables contribute a great deal to the model.

The "Partial R-Square" value tells us how much variation in the predicted response is attributed to each feature variable. The $height$ variable contributes $\frac{0.9798}{0.9879} \times 100\% = 99.18\%$ of the variability, while $length_1$ contributes  $\frac{0.0055}{0.9879} \times 100\% = 0.55\%$ and $width$ contributes $\frac{0.0055}{0.9879} \times 100\% = 0.25\%$.

The F-value reinforces this notion by performing a comparison of the partial $R^2$ value against the variance in the error distribution. The p-value derived from the F-value demonstrates how significant the variable is to the model. All variables are found to be highly significant.

Notice that the $width$ variable contributes very little at all. Let's see if that feature is eliminated using backwards selection:

In [29]:
proc reg data = perch plots = none;
    model weight = length1 length2 length3 height width / selection = backward;
run;

The backwards selection algorithm eliminates $length_3$ and $length_2$, leaving us with the same variables from before. This is due to the fact that the least statistically significant variable is eliminated in the presence of the other variables. This elimination is repeated until all variables are significant.

In the first two iterations $length_1$ is found to be insignificant, but is more significant than either $length_2$ or $length_3$. Once those two are eliminated, $length_1$ is significant in the presence of $height$ and $width$ and is allowed to stay. This makes intuitive sense because $length_1$, $length_2$, and $length_3$ carry very similar information, making only one of them necessary.

Now, let's look at stepwise selection:

In [30]:
proc reg data = perch plots = none;
    model weight = length1 length2 length3 height width / selection = stepwise;
run;

Once again, this produces similar results to the forward and backward elimination steps. Since stepwise elimination starts the same way as its forward counterpart, we should expect similar results. This time, however, stepwise elimination deletes any variables deemed insigificant in the presence of a newly added variable.

To prevent model overfitting, let's look at the $R^2_{adj}$ selection:

In [31]:
proc reg data = perch plots = none;
    model weight = length1 length2 length3 height width / selection = adjrsq;
run;

Here, it is seen that $length_1$, $height$ and $width$ give the highest $R^2_{adj}$ value. Switching $length_1$ with $length_2$ or $length_3$ gives similar $R^2_{adj}$ values, which isn't surprising given that they convey almost the exact same information.

Finally, let's use Mallows' $C_p$ selection:

In [33]:
proc reg data = perch plots = none;
    model weight = length1 length2 length3 height width / selection = cp;
run;

The Mallows' $C_p$ statistic prevents overfitting by penalizing the addition of new feature variables. Its formula is this:

\begin{equation}
C_p = \frac{SSE}{s^2} - N + 2P
\end{equation}

where the $SSE$ is found for a particular subset $P$ of feature variables. As more feature variables are addded the $C_p$ value increases. Therefore, the best model would be one that minimizes $C_p$. In this case, the following variables generate a model with the lowest $C_p$:
* $length_1$
* $height$
* $width$

which are the same three that are chosen throughout this example.

The model, as chosen by the selection methods, can be written as:
\begin{equation}
\text{weight} = 75.91 - 16.96 length_1 + 194.65 height + 152.49 width
\end{equation}

It is interesting to see that the $length_1$ of a fish is inversely proportional to the weight.

As a side note, I believe that $length_1$ is chosen over the other two lengths because the tail does not add much to the weight of a fish. The length of the body of the fish explains the weight more than the length of the body with the tail. Therefore, $length_1$ gives the minimal amount of information required to explain the weight of the fish.

## Summary

* model selection methods help sort through many feature variables to isolate ones which will significantly benefit a model